In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets,transforms
import random
import torch.nn.functional as F
import math

In [2]:
class Net(nn.Module):
    #nn.module is the parent class for all pytorch models, it has some inbuilt functionalities which calculates gradients
    
    def __init__(self):# constructor
        
        super(Net,self).__init__()
        
        # 1 input layer of 784(28*28 flattened) and a hidden layer with 30 neuron and finally output layer of 10 neurons
        self.fc1=nn.Linear(784,30)
        #self.drop=nn.Dropout(0.2)
        self.fc2=nn.Linear(30,1)
        
    def forward(self,x):
        #x-shape [64,784] [batchsize,inputsize]
        #layer-1
        l1=self.fc1(x)# l1-shape [64,30]
        
        #activation function-1
        al1=torch.relu(l1)#al1-shape[64,30]
        
        #layer-2
        l2=self.fc2(al1)#l2-shape [64,10]
        
        #activation dunction-2
        al2=torch.relu(l2)#al2-shape[64,10] which is our output
        
        return l2

In [3]:
def train(model,use_cuda,train_loader,optimizer,epoch):
    
    model.train()#tell the model to prepare for training
    
    for batchid,(data,target) in enumerate(train_loader): # get the batch
        
        #converting the data to [batch_size,784] from [batch_size,1,28,28]
        data = data.reshape([data.shape[0], 784]) # [batch_size,784]
        
        if use_cuda:
            data,target=data.cuda(),target.cuda() #setting the data to GPU if using Gpus
            
        optimizer.zero_grad()#setting the cumulative gradients to zero
        output=model(data)#forward pass through the model
        output=(torch.flatten(output))
        loss=torch.sqrt(torch.mean((output-target)**2))#calculating loss MSE
        loss.backward()#calculating gradients of the model
        optimizer.step()#updating model parameters . this step doesnt remove stored gradients
        
        if batchid % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batchid * len(data), len(train_loader.dataset),
            100. * batchid / len(train_loader), loss.item()))

In [4]:
def test(model,use_cuda,test_loader):
    
    model.eval()#to tell the model to be ready for evaluation
    
    test_loss=0
    correct=0
    
    with torch.no_grad():#to ensure gradients are not calculated as calculating gradients is not required for testing
        for data,target in test_loader:# getting the batch
            
            #converting the data to [batch_size,784] from [batch_size,1,28,28]
            data=data.reshape([data.shape[0],784])
            
            if use_cuda:
                data,target=data.cuda(),target.cuda()
                
            output=model(data)#forward pass
            output=torch.flatten(output)
            test_loss+=math.sqrt(torch.sum((output-target)**2))#sum up batch loss
            
            pred=torch.round(output)
            correct += pred.eq(target.view_as(pred)).sum().item()#getting total number of correct samples
            
        test_loss/=len(test_loader.dataset)
        print(test_loss,100*correct/len(test_loader.dataset))#accuracy=total correct/total samples

In [5]:
def seed(seed_value):
    #this function removes randomness and makes everything deterministic
    #here we set the seed for torch.cuda,torch,numpy and random.
    #torch.cuda.manual_seed_all(seed_value) ,if we are using multi-GPU then we should use this to set the seed.
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed(seed_value)
    np.random.seed(seed_value)
    random.seed(seed_value)

In [6]:
def main():
    use_cuda=False #if we have GPUs then we can set this true else False and program runs on the cpu
    seed(0) #used to fix the randomness of the code.
    
    #converting the dataset to tensor and normalizing the data by subtracting mean and dividing it by standard deviation,
    #normalizing results in the faster converging of model to optimal values.
    
    # x[0-255] => [0-1]
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,),(0.3081,)) #((mean),(std))    (x-mean)/std
    ])
    
    data1=datasets.MNIST('../data',download=True,train=True,transform=transform)#getting the train dataset, shape=[60000,28,28]
    data2=datasets.MNIST('../data',train=False,transform=transform)#getting the test dataset. shape=[10000,28,28]
    
    #loading data using torch's inbuilt dataloader
    train_loader=torch.utils.data.DataLoader(data1,num_workers=2,batch_size=64,shuffle=True)#getting train dataloader
    test_loader=torch.utils.data.DataLoader(data2,num_workers=2,batch_size=1000,shuffle=False)#getting test dataloader
    
    model=Net()
    
    if use_cuda:
        model=model.cuda() # if we use GPUs we can put the model weights on GPU 
        
    optimizer=optim.Adam(model.parameters(),lr=0.001)#choosing the optimizer and setting the learning rate.
    
    for epoch in range(1,31):
        
        train(model,use_cuda,train_loader,optimizer,epoch)
        test(model,use_cuda,test_loader)

In [7]:
if __name__=="__main__":
    main()

Train Epoch: 1 [0/60000 (0%)]	Loss: 5.177732
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.722982
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.480200
Train Epoch: 1 [19200/60000 (32%)]	Loss: 1.686641
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.748472
Train Epoch: 1 [32000/60000 (53%)]	Loss: 1.448788
Train Epoch: 1 [38400/60000 (64%)]	Loss: 1.699176
Train Epoch: 1 [44800/60000 (75%)]	Loss: 1.347685
Train Epoch: 1 [51200/60000 (85%)]	Loss: 1.491495
Train Epoch: 1 [57600/60000 (96%)]	Loss: 1.445682
0.0408000806262085 38.67
Train Epoch: 2 [0/60000 (0%)]	Loss: 1.505798
Train Epoch: 2 [6400/60000 (11%)]	Loss: 1.024180
Train Epoch: 2 [12800/60000 (21%)]	Loss: 1.126535
Train Epoch: 2 [19200/60000 (32%)]	Loss: 1.040171
Train Epoch: 2 [25600/60000 (43%)]	Loss: 1.436952
Train Epoch: 2 [32000/60000 (53%)]	Loss: 1.223148
Train Epoch: 2 [38400/60000 (64%)]	Loss: 1.290226
Train Epoch: 2 [44800/60000 (75%)]	Loss: 1.401865
Train Epoch: 2 [51200/60000 (85%)]	Loss: 1.020927
Train Epoch: 2 [57600/60000 (96%)]	Lo

Train Epoch: 16 [44800/60000 (75%)]	Loss: 1.060385
Train Epoch: 16 [51200/60000 (85%)]	Loss: 1.033818
Train Epoch: 16 [57600/60000 (96%)]	Loss: 0.968709
0.03360111610644885 50.12
Train Epoch: 17 [0/60000 (0%)]	Loss: 1.270256
Train Epoch: 17 [6400/60000 (11%)]	Loss: 0.972779
Train Epoch: 17 [12800/60000 (21%)]	Loss: 1.150280
Train Epoch: 17 [19200/60000 (32%)]	Loss: 1.024266
Train Epoch: 17 [25600/60000 (43%)]	Loss: 0.831726
Train Epoch: 17 [32000/60000 (53%)]	Loss: 1.542159
Train Epoch: 17 [38400/60000 (64%)]	Loss: 0.784464
Train Epoch: 17 [44800/60000 (75%)]	Loss: 1.009512
Train Epoch: 17 [51200/60000 (85%)]	Loss: 1.060637
Train Epoch: 17 [57600/60000 (96%)]	Loss: 0.858736
0.03398171533882785 49.14
Train Epoch: 18 [0/60000 (0%)]	Loss: 0.965071
Train Epoch: 18 [6400/60000 (11%)]	Loss: 0.803692
Train Epoch: 18 [12800/60000 (21%)]	Loss: 0.923144
Train Epoch: 18 [19200/60000 (32%)]	Loss: 0.852666
Train Epoch: 18 [25600/60000 (43%)]	Loss: 1.102799
Train Epoch: 18 [32000/60000 (53%)]	Loss: 

In [8]:
import tensorflow
from keras.models import Sequential
from keras.layers.core import Dense,Dropout,Activation
from keras.datasets import mnist
from keras.utils import np_utils

2022-07-20 12:18:33.203218: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-07-20 12:18:33.206012: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-20 12:18:33.206024: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [9]:
(xt,yt),(xte,yte)=mnist.load_data()

In [10]:
xt=xt.reshape(60000,784).astype('float32')
xte=xte.reshape(10000,784).astype('float32')
xt/=255
xte/=255

In [11]:
model=Sequential()                        #Linear stacking of layers

model.add(Dense(512,input_shape=(784,)))  #First layer with 512 nodes
model.add(Activation('relu'))#Relu activation function
model.add(Dropout(0.2)) 

model.add(Dense(50))                      #second layer with 50 nodes
model.add(Activation('relu'))             #relu activation layer
model.add(Dropout(0.2))                   #20% dropout of randomly selected nodes

model.add(Dense(1))                      #Final layer with 10 nodes and
#model.add(Activation('relu'))    

2022-07-20 12:18:38.431985: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-07-20 12:18:38.432027: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-20 12:18:38.432045: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (localhost.localdomain): /proc/driver/nvidia/version does not exist
2022-07-20 12:18:38.432240: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 activation (Activation)     (None, 512)               0         
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 50)                25650     
                                                                 
 activation_1 (Activation)   (None, 50)                0         
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 5

In [13]:
model.compile(loss=tensorflow.keras.losses.MeanSquaredError(), optimizer='adam', metrics=['accuracy'])

In [14]:
model.fit(x=xt,y=yt,batch_size=64,epochs=300)

Epoch 1/300
938/938 [==============================] - 3s 3ms/step - loss: 2.1085 - accuracy: 0.1752
Epoch 2/300
938/938 [==============================] - 2s 3ms/step - loss: 1.1946 - accuracy: 0.1954
Epoch 3/300
938/938 [==============================] - 2s 3ms/step - loss: 0.9719 - accuracy: 0.1946
Epoch 4/300
938/938 [==============================] - 2s 3ms/step - loss: 0.8519 - accuracy: 0.1945
Epoch 5/300
938/938 [==============================] - 2s 3ms/step - loss: 0.7423 - accuracy: 0.1913
Epoch 6/300
938/938 [==============================] - 2s 3ms/step - loss: 0.6587 - accuracy: 0.1899
Epoch 7/300
938/938 [==============================] - 2s 3ms/step - loss: 0.5973 - accuracy: 0.1890
Epoch 8/300
938/938 [==============================] - 2s 3ms/step - loss: 0.5502 - accuracy: 0.1902
Epoch 9/300
938/938 [==============================] - 2s 3ms/step - loss: 0.5154 - accuracy: 0.1889
Epoch 10/300
938/938 [==============================] - 2s 3ms/step - loss: 0.4761 - accura

938/938 [==============================] - 2s 3ms/step - loss: 0.1728 - accuracy: 0.1906
Epoch 82/300
938/938 [==============================] - 2s 3ms/step - loss: 0.1650 - accuracy: 0.1910
Epoch 83/300
938/938 [==============================] - 2s 3ms/step - loss: 0.1668 - accuracy: 0.1903
Epoch 84/300
938/938 [==============================] - 2s 3ms/step - loss: 0.1699 - accuracy: 0.1901
Epoch 85/300
938/938 [==============================] - 2s 3ms/step - loss: 0.1703 - accuracy: 0.1917
Epoch 86/300
938/938 [==============================] - 2s 3ms/step - loss: 0.1638 - accuracy: 0.1917
Epoch 87/300
938/938 [==============================] - 3s 3ms/step - loss: 0.1728 - accuracy: 0.1910
Epoch 88/300
938/938 [==============================] - 2s 3ms/step - loss: 0.1665 - accuracy: 0.1915
Epoch 89/300
938/938 [==============================] - 2s 3ms/step - loss: 0.1710 - accuracy: 0.1911
Epoch 90/300
938/938 [==============================] - 2s 3ms/step - loss: 0.1648 - accuracy: 

938/938 [==============================] - 2s 3ms/step - loss: 0.1429 - accuracy: 0.1924
Epoch 161/300
938/938 [==============================] - 3s 3ms/step - loss: 0.1453 - accuracy: 0.1912
Epoch 162/300
938/938 [==============================] - 2s 3ms/step - loss: 0.1515 - accuracy: 0.1908
Epoch 163/300
938/938 [==============================] - 2s 3ms/step - loss: 0.1473 - accuracy: 0.1918
Epoch 164/300
938/938 [==============================] - 2s 3ms/step - loss: 0.1505 - accuracy: 0.1920
Epoch 165/300
938/938 [==============================] - 2s 3ms/step - loss: 0.1477 - accuracy: 0.1922
Epoch 166/300
938/938 [==============================] - 2s 3ms/step - loss: 0.1468 - accuracy: 0.1915
Epoch 167/300
938/938 [==============================] - 2s 3ms/step - loss: 0.1449 - accuracy: 0.1908
Epoch 168/300
938/938 [==============================] - 2s 3ms/step - loss: 0.1428 - accuracy: 0.1922
Epoch 169/300
938/938 [==============================] - 2s 3ms/step - loss: 0.1496 - a

938/938 [==============================] - 2s 3ms/step - loss: 0.1434 - accuracy: 0.1911
Epoch 240/300
938/938 [==============================] - 2s 3ms/step - loss: 0.1372 - accuracy: 0.1910
Epoch 241/300
938/938 [==============================] - 2s 3ms/step - loss: 0.1374 - accuracy: 0.1914
Epoch 242/300
938/938 [==============================] - 2s 3ms/step - loss: 0.1351 - accuracy: 0.1926
Epoch 243/300
938/938 [==============================] - 2s 3ms/step - loss: 0.1394 - accuracy: 0.1923
Epoch 244/300
938/938 [==============================] - 2s 3ms/step - loss: 0.1377 - accuracy: 0.1913
Epoch 245/300
938/938 [==============================] - 2s 3ms/step - loss: 0.1443 - accuracy: 0.1918
Epoch 246/300
938/938 [==============================] - 2s 3ms/step - loss: 0.1365 - accuracy: 0.1922
Epoch 247/300
938/938 [==============================] - 2s 3ms/step - loss: 0.1394 - accuracy: 0.1923
Epoch 248/300
938/938 [==============================] - 2s 3ms/step - loss: 0.1375 - a

In [15]:
score = model.evaluate(xte,yte)
print('Test score:', score[0])
print('Test accuracy:', score[1])

313/313 [==============================] - 0s 951us/step - loss: 0.3540 - accuracy: 0.2053
Test score: 0.3539540767669678
Test accuracy: 0.2053000032901764


In [31]:
preds=model.predict(xte)

313/313 [==============================] - 0s 830us/step


In [33]:
rmse=0
for i in range(len(preds)):
    rmse+=(preds[i]-yte[i])**2
print((rmse/len(preds))**0.5)

[0.594939]


In [26]:
preds=np.round(preds)

In [27]:
acc=0
for i in range(len(preds)):
    if preds[i]==yte[i]:
        acc+=1
print(acc/len(preds))

0.9259


In [28]:
from sklearn import metrics

In [29]:
print(metrics.precision_score(y_true=yte,y_pred=preds,labels=[1],average=None))

[0.97137746]


In [30]:
print(metrics.recall_score(y_true=yte,y_pred=preds,labels=[1],average=None))

[0.95682819]


In [36]:
model.save('regmodel')

INFO:tensorflow:Assets written to: regmodel/assets
